# Running the simulation
## Setup Runner

In [ ]:
print("--- RUNNING MOVEMENT SIMULATION ---")
import warnings

warnings.filterwarnings("ignore")

from agent_torch.populations import sample2
import simulation
from agent_torch.core.environment import envs


def run_movement_simulation():
    """Run the movement simulation using our example model."""
    print("\n=== Running Movement Simulation ===")

    # Create the runner using envs.create
    print("\nCreating simulation runner...")
    runner = envs.create(model=simulation, population=sample2)

    # Get simulation parameters from config
    sim_steps = runner.config["simulation_metadata"]["num_steps_per_episode"]
    num_episodes = runner.config["simulation_metadata"]["num_episodes"]

    print(f"\nSimulation parameters:")
    print(f"- Steps per episode: {sim_steps}")
    print(f"- Number of episodes: {num_episodes}")
    print(f"- Number of agents: {runner.config['simulation_metadata']['num_agents']}")

    # Run all episodes
    print("\nRunning simulation episodes...")
    for episode in range(num_episodes):
        print(f"\nEpisode {episode + 1}/{num_episodes}")

        # Reset state at the start of each episode
        if episode > 0:
            runner.reset()

        # Run one episode
        runner.step(sim_steps)

        # Print statistics
        final_state = runner.state
        edge_progress = final_state["agents"]["citizens"]["edge_progress"]
        #print(positions)
        #print(f"- Average position: {positions.mean(dim=0)}")

    print("\nSimulation completed!")
    return runner


if __name__ == "__main__":
    runner = run_movement_simulation()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np
import torch
import matplotlib as mpl

mpl.rcParams['animation.embed_limit'] = 50  # allow large inline animations

def visualize_simulation(runner):
    """
    Visualize the agent-based movement simulation.
    Uses node positions and edge lengths from the environment.
    """
    # === Load graph and positions ===
    adjacency_matrix = np.array(runner.state["environment"]["graph"])
    positions = np.array(runner.state["environment"]["node_positions"])

    num_nodes = len(positions)

    # Create weighted undirected graph
    G = nx.Graph()
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if adjacency_matrix[i][j] > 0:
                G.add_edge(i, j, weight=adjacency_matrix[i][j])

    # Use provided node coordinates
    pos = {i: (positions[i][0], positions[i][1]) for i in range(num_nodes)}

    # === Setup plot ===
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_aspect("equal", adjustable="datalim")
    ax.set_title("Agent Movement Simulation")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    # Draw graph structure
    nx.draw_networkx_edges(
        G, pos, ax=ax, edge_color="gray", width=1.0, alpha=0.5
    )
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=300, node_color="lightblue")
    nx.draw_networkx_labels(G, pos, font_size=9)

    # === Initialize agents ===
    current_edge = np.array(runner.state["agents"]["citizens"]["current_edge"])
    edge_progress = np.array(runner.state["agents"]["citizens"]["edge_progress"])
    num_agents = len(current_edge)

    # Random fixed colors for each agent
    agent_colors = np.random.rand(num_agents, 3)

    # Initialize scatter plot for agents
    agent_positions = np.zeros((num_agents, 2))
    agent_scat = ax.scatter(
        agent_positions[:, 0],
        agent_positions[:, 1],
        s=40,
        color=agent_colors,
        edgecolors="k",
        linewidths=0.3
    )

    # === Animation functions ===
    def init():
        agent_scat.set_offsets(np.zeros((num_agents, 2)))
        return (agent_scat,)

    def update(frame):
        # Advance simulation by one step
        runner.step(1)

        current_edge = np.array(runner.state["agents"]["citizens"]["current_edge"])
        edge_progress = np.array(runner.state["agents"]["citizens"]["edge_progress"])

        new_positions = []
        for i in range(num_agents):
            start_node = int(current_edge[i][0])
            end_node = int(current_edge[i][1])
            progress = float(edge_progress[i])

            x_start, y_start = pos[start_node]
            x_end, y_end = pos[end_node]

            x = x_start + (x_end - x_start) * progress
            y = y_start + (y_end - y_start) * progress
            new_positions.append((x, y))

        agent_scat.set_offsets(np.array(new_positions))
        ax.set_title(f"Simulation Step {frame + 1}")
        return (agent_scat,)

    ani = animation.FuncAnimation(
        fig,
        update,
        init_func=init,
        frames=600,
        interval=100,
        blit=True,
        repeat=False
    )

    return HTML(ani.to_jshtml())

In [ ]:
visualize_simulation(runner)

## Plot per step

In [ ]:
import matplotlib.pyplot as plt

def plot_positions_per_step(runner):
    print("\n--- PLOTTING AGENT POSITIONS ---")
    for step_idx, step_list in enumerate(runner.state_trajectory):
        # Each step is a list with one dict
        state = step_list[0]

        # Get positions
        positions = state['agents']['citizens']['position']  # tensor [num_agents, 2]
        positions = positions.detach().cpu().numpy()                 # convert to numpy

        plt.figure(figsize=(6,6))
        plt.scatter(positions[:,0], positions[:,1], c='blue', label='Citizens')
        plt.xlim(0, state['environment']['bounds'][0])
        plt.ylim(0, state['environment']['bounds'][1])
        plt.title(f"Step {step_idx + 1}")
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.legend()
        plt.grid(True)
        plt.show()

# Call after running the simulation
plot_positions_per_step(runner)

## Animate state over steps

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import torch

def animate_positions(runner):
    print("\n--- CREATING ANIMATION ---")

    # Extract bounds
    grid_width = runner.state_trajectory[0][0]['environment']['bounds'][0].item()
    grid_height = runner.state_trajectory[0][0]['environment']['bounds'][1].item()

    # Prepare figure
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xlim(0, grid_width)
    ax.set_ylim(0, grid_height)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True)
    scatter = ax.scatter([], [], c='blue', label='Citizens')
    ax.legend()

    # Preprocess positions for all steps
    all_positions = []
    for step_list in runner.state_trajectory:
        state = step_list[0]
        positions = state['agents']['citizens']['position'].detach().cpu().numpy()
        all_positions.append(positions)

    # Update function
    def update(frame):
        positions = all_positions[frame]
        scatter.set_offsets(positions)
        ax.set_title(f"Step {frame + 1}")
        return scatter,

    # Create animation
    anim = FuncAnimation(fig, update, frames=len(all_positions), interval=300, blit=True)

    # Display in Jupyter
    return HTML(anim.to_jshtml())

# Call in notebook
animate_positions(runner)